In [14]:
import matplotlib.pyplot as plt
import numpy as np
import random

## Exploring reward funcion options

## **Linear - user score constant**

This is an weighted average function that will assign constant weights to a multiple set of parameters and come up with a score that would be then used to rank users and assign token rewards.

### **Function**

In [6]:
def linear_no_userscore(parameters_weights:list,
                        total_number_of_views:int, 
                        number_of_unique_viewers:int,
                        number_of_prompts_created_from_content:int,
                        views_by_user_score:dict,
                        ):
    '''
    total_number_of_views: total numbers of queries that saw/used that content, 
    number_of_unique_viewers: number of unique users that saw/used the content through the queries,
    number_of_prompts_created_from_content: number of other user's content created on top of the user content,
    views_by_user_score: {user_score: total_number_of_views}
    * parameters_weights: list of weights for each pameters, range should be [0, 100] and parameter_weights.sum() == 100
                            [ w_total_views, w_unique_view, w_prompts_created, w_views_by_score] in this order
    
    returns: user final score for the epoch
    '''
    # assigning weights
    w_total_views = parameters_weights[0]
    w_unique_view = parameters_weights[1]
    w_prompts_created = parameters_weights[2]
    w_views_by_score = parameters_weights[3]

    # views by score
    # weighted average of points 
    total_scores = 0
    score_view_products = 0
    for score in views_by_user_score.keys():
        score_view_products += score * views_by_user_score[score]
        total_scores += score
    
    weighted_user_score = score_view_products / total_scores

    final_result = (total_number_of_views * w_total_views) \
                    + (number_of_unique_viewers * w_unique_view) \
                    + (number_of_prompts_created_from_content * w_prompts_created ) + \
                    + (w_views_by_score * weighted_user_score)

    return final_result

In [5]:
function_to_test = linear_no_userscore

### **Tests**

In [ ]:
class TestGenerator:
    def __init__(
                self,
                number_of_users_to_generate:int,
                total_views_average:int,
                total_unique_views_average:int,
                total_prompts_created_average:int,
                total_views_perc_extreme:float = 0.1,
                total_views_has_extreme:bool = False,
                total_views_compact:bool = True,
                compact_factor:float = 0.1,
                extreme_factor:int = 3
                ):
        '''
        compact_factor: the percentage where the random values will be generate from: average_Value * +- compact_factor
        extreme_factor: factor by what I multiply when values are not compacted to create the range from [1, averege * max I want for the metric to go]
        '''
        self.compact_factor = compact_factor
        self.number_of_users_to_genereate = number_of_users_to_generate
        self.total_views_average = total_views_average
        self.total_unique_views_average = total_unique_views_average
        self.total_prompts_created_average = total_prompts_created_average
        self.total_views_has_extreme = total_views_has_extreme
        self.total_views_compact = total_views_compact

    def generate_users(self):
        simulated_users = []
        for x in range(self.number_of_users_to_genereate):
            user_data = {'id': x}
            user_data['total_views'] = self.generate_total_views()

            simulated_users.append(user_data)
            
            

    
    def generate_total_views(self):
        if self.total_views_compact:
            delta = self.total_unique_views_averagecompact_factor * self.compact_factor
            return random.randrange(self.total_views_average - delta, self.total_views_average + delta)
        else:
            return  random.randrange( 1 , self.total_unique_views_average * self.extreme_factor)



        

In [17]:
random.randrange(0,10)

8

#### Similar users

#### Testing extremes 

#### Similar user scores

#### Similar user scores

#### Similar user scores